# How to calculate and visualize Aerosol Optical Depths (AOD) from MISR

## Summary

## Note

By Cheyenne Land, Walt Baskin

Tested using Python 3.8.10

## 1. Setup

In [ ]:
import h5py
import numpy as np
from matplotlib import pyplot as PLT
import cartopy.crs as ccrs
import earthaccess
import os

home_dir = os.environ['HOME']
MISR_dir = os.path.abspath(f"{home_dir}/MISR")
if not os.path.exists(MISR_dir):
    os.makedirs(MISR_dir)

%reload_ext autoreload
%autoreload 2

## 2. Search for data using earthaccess

In [ ]:
earthaccess.login()
short_name = 'MIL3YAEN'
version = '004'
collection = short_name + '_' + version
results = earthaccess.search_data(
    short_name= short_name,
    version = version,
    provider = 'LARC_CLOUD',
    temporal = ("2019-12", "2020-12")
)
print(f'{len(results[:])} file(s) found.')
print(results)

## 3. Download data

In [31]:
for r in results[:]:
    file_name = r['umm']['DataGranule']['Identifiers'][0]['Identifier']
    try:
        os.makedirs(f'{MISR_dir}/{collection}')
    except FileExistsError: 
        continue
    finally:
        if os.path.exists(f'{MISR_dir}/{collection}/{file_name}') == True:
            print(f'The file, {file_name} already exists. \nLocated here: ~/MISR/{collection}/{file_name}')
        else:
            earthaccess.download(r, f'{MISR_dir}/{collection}')
            print(f'File, {file_name}, downloaded here: ~/MISR/{collection}')

QUEUEING TASKS | : 100%|██████████| 1/1 [00:00<00:00, 1905.64it/s]
PROCESSING TASKS | : 100%|██████████| 1/1 [00:09<00:00,  9.17s/it]
COLLECTING RESULTS | : 100%|██████████| 1/1 [00:00<00:00, 11781.75it/s]


File, MISR_AM1_CGAS_2020_F15_0032.nc, downloaded here: ~/MISR/MIL3YAEN_004


QUEUEING TASKS | : 100%|██████████| 1/1 [00:00<00:00, 1919.59it/s]
PROCESSING TASKS | : 100%|██████████| 1/1 [00:08<00:00,  8.93s/it]
COLLECTING RESULTS | : 100%|██████████| 1/1 [00:00<00:00, 10512.04it/s]

File, MISR_AM1_CGAS_2021_F15_0032.nc, downloaded here: ~/MISR/MIL3YAEN_004


## 4. Generate plot

In [ ]:
# Open and read file
for file in os.listdir(f'{MISR_dir}/{collection}'):
    if file.endswith(".nc"):
        data_file = os.path.join(f'{MISR_dir}/{collection}', file)
        with h5py.File(data_file, mode='r') as f:
            optical_depth = 'Aerosol_Parameter_Average/Absorbing_Aerosol_Optical_Depth_Per_Band'
            data_optical_depth = f[optical_depth][:].astype(np.float64)
            
            #slice data
            # by [Lat, Lon, Optical Depth Range, Band]

            # Before conducting the histogram equalizatoin, slice the data with
            # a value in each dimension of your choosing

            # Optical Depth Range: 0 = All, 1 = <0.05, 2 = 0.05 - 0.15, 3 = 0.15 - 0.25, 4 = 0.25 - 0.4,
            # 5 = 0.4 - 0.6, 6 = 0.6 - 0.8, 7 = 0.8 - 1.0, 8 = > 1.0

            # MISR camera bands: 0 = Blue (443 nm), 1 = Green (555 nm), 2 = Red (670 nm), nir (867).

            # For more informaion go to [https://asdc.larc.nasa.gov/documents/misr/dps/dps.html]

            #In this case we have each band at the optical depth of < 0.05
            blueband = data_optical_depth[:,:,1,0]
            greenband = data_optical_depth[:,:,1,1]
            redband = data_optical_depth[:,:,1,2]

            #Stack the blue, green and red bands to use the histogram equalization function
            data = np.dstack((blueband,greenband,redband))
            
            #Histogram Equalization
            # Histogram Equalization is a technique that spreads out the most frequent intensity values
            # to improve the contrast in images (Sudhakar). The histogram equalization process also relies on the
            # cumulative distributive function (cdf). The cdf describes the probabliity that a variable takes a value less than or equal to x (COSTE).
            # Using this function for the MISR Aerosol Optical Depth script will allow the user to have a better visualization of the data.

            # Sudhakar, Shreenidhi. “Histogram Equalization.” Medium, Towards Data Science, 30 Jan. 2021, towardsdatascience.com/histogram-equalization-5d1013626e64.

            # COSTE, Arthur. “Project 1 : HISTOGRAMS.” Image Processing, 5 Sept. 2012, www.sci.utah.edu/~acoste/uou/Image/project1/Arthur_COSTE_Project_1_report.html.
            
            nbr_bins = 256
            data_min = 0.0
            data_max = 1.0
            data = np.where(data < data_min, data_min, data)
            data = np.where(data > data_max, data_max, data)
            ''' histogram equalization '''
            datahist,bins = np.histogram(data.flatten(),nbr_bins, density=True)
            ''' cumulative distribution function '''
            cdf = datahist.cumsum()
            cdf_normalized = cdf / cdf.max()
            ''' use linear interpolation of cdf to find new pixel values '''
            out_data = np.interp(data.flatten(),bins[:-1],cdf_normalized)
            ''' scale between 0 and 1 '''
            out_data_min = np.min(out_data)
            out_data_max = np.max(out_data)
            data_range = out_data_max - out_data_min
            ''' normalize '''
            out_data = ((out_data - out_data_min)/data_range)
            oda_data = out_data.reshape(data.shape)

            #Turn fill values (-9999) to NaN
            masked_data = np.ma.masked_where(oda_data <= 0, oda_data)
            final_oda_data = oda_data.copy()
            final_oda_data[masked_data <= 0] = np.nan

            #Convert array type from float64 to the datas original data type, float32
            final32 = np.float32(final_oda_data)

            #Select a band to plot
            #0 = Blue (443 nm), 1 = Green (555 nm), 2 = Red (670 nm)
            greenbandfinal = final32[:,:,1]

            #Plot figure
            PLT.figure(figsize=(7.20,3.60))
            lons = np.linspace(-179.75,179.75, 720)
            lats = np.linspace(89.75,-89.75, 360)
            ax = PLT.axes(projection=ccrs.PlateCarree())
            im = PLT.contourf(lons, lats, greenbandfinal, 300, cmap=PLT.get_cmap('jet'),
                        transform=ccrs.PlateCarree())
            ax.coastlines()
            PLT.title(f'Absorbing Aerosol Optical Depth at < 0.05 and 555nm\n {file}', fontsize=8)
            cb = PLT.colorbar(im, ticks = [0 , 1 ] , shrink=0.83)
            cb.set_label('Absorbing Aerosol Optical Depth', fontsize=8)

            try: 
                if os.path.exists(f'{MISR_dir}/{collection}/{file}.jpg') == True:
                    print(f'This plot already exits: {MISR_dir}/{collection}/{file}.jpg')
                else:
                    PLT.savefig(f'{MISR_dir}/{collection}/{file}.jpg', dpi=200)
                    print(f'Plot saved here: ~/MISR/{collection}/{file}.png')
            except FileExistsError:
                continue
        
            PLT.close()
            print('Removing data file...')
            os.remove(data_file)
